# 蛋白质比较

## 目标和前提条件

在本示例中，我们将向您展示如何使用数学优化来解决蛋白质比较问题。您将学习如何使用Gurobi Python API将这个问题（涉及测量两种蛋白质的相似性）建模为二次分配问题，并使用Gurobi优化器找到其最优解。

这个模型是H. Paul Williams的《数学规划模型构建》第五版中的第29个示例，见第290-291页和第345页。

这是一个高级建模示例，我们假设您已经了解Python和Gurobi Python API，并且具有构建数学优化模型的高级知识。通常，这些示例的目标函数和/或约束条件较为复杂，需要使用Gurobi Python API的高级功能。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

In [ ]:
%pip install gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## 问题描述

这个问题基于Forrester和Greenberg（2008）论文中讨论的一个问题。它关注于测量两种蛋白质的相似性。蛋白质可以用图形表示，其中氨基酸由节点表示，当两个氨基酸之间的距离在阈值范围内时，就存在边连接。这种图形表示被称为蛋白质的接触图。

给定表示两种蛋白质的接触图，我们希望在每个图中找到最大的（以对应边的数量衡量）同构子图。每个蛋白质中的氨基酸是有序的。我们需要在每个子图中保持这种顺序，这意味着在比较中不能有交叉。这在下图中有所说明。

![交叉](crossover.PNG)

如果在第一个蛋白质的接触图中 $i < k$，那么在第二个蛋白质中不能有 $l < j$，如果在比较中 $i$ 要与 $j$ 关联，而 $k$ 要与 $l$ 关联。下图给出了两个小型接触图之间的比较，导致五个对应边。

![比较](comparison.PNG)

目标是比较以下图形所给出的接触图。

第一个蛋白质的映射：

![图1](map1.PNG)

第二个蛋白质的映射：

![图2](map2.PNG)

## 模型构建

第一个蛋白质的映射由图 $G_{1} = (N_{1},E_{1})$ 表示，第二个蛋白质的映射由图 $G_{2} = (N_{2},E_{2})$ 表示

### 集合和索引

$i,k \in N_{1} =\{1,2,...,9\}$：图 $G_{1}$ 中的节点，即第一个蛋白质中的氨基酸。

$E_{1} = \{(1,2),(2,9),(3,4),(3,5),(5,6),(6,7),(7,9),(8,9) \}$：图 $G_{1}$ 中的边。

$j,l \in N_{2} =\{1,2,...,11\}$：图 $G_{2}$ 中的节点，即第二个蛋白质中的氨基酸。

$E_{2} = \{(1,4),(2,3),(4,6),(4,7),(5,6),(6,8),(7,8),(7,10),(9,10),(10,11) \}$：图 $G_{2}$ 中的边。

### 决策变量

$\text{map}_{i,j} = x_{i,j} = 1$，当且仅当 $G_{1}$ 中的节点 $i$ 与 $G_{2}$ 中的节点 $j$ 匹配。

$w_{i,j,k,l} = x_{i,j}*x_{k,l} = 1$，当且仅当 $E_{1}$ 中的边 $(i,k)$ 与 $E_{2}$ 中的边 $(j,l)$ 匹配。

### 约束条件

**$G_{1}$ 匹配约束**：$G_{1}$ 中的节点最多只能与 $G_{2}$ 中的一个节点匹配。

$$
\sum_{i \in N_{1} } x_{i,j} \leq 1 \quad \forall j \in N_{2}
$$

**$G_{2}$ 匹配约束**：$G_{2}$ 中的节点最多只能与 $G_{1}$ 中的一个节点匹配。

$$
\sum_{j \in N_{2} } x_{i,j} \leq 1 \quad \forall i \in N_{1}
$$

**边匹配约束**：如果边 $(i,k)$ 和 $(j,l)$ 匹配，则相应的节点也必须匹配。

$$
w_{i,j,k,l} \leq x_{i,j}, \; w_{i,j,k,l} \leq x_{k,l} \quad \forall 
(i,j,k,l) \in ijkl = \{ i,k \in N_{1}, j,l \in N_{2}: (i,k) \in E_{1}, (j,l) \in E_{2} \}
$$

**无交叉约束**：不能存在交叉。

$$
x_{i,j} + x_{k,l} \leq 1 \quad \forall 
(i,j,k,l) \in ijklx = \{ (i,j,k,l) \in ijkl: i < k \in N_{1}, j > l \in N_{2} \}
$$

### 目标函数
目标是最大化边匹配的数量。

$$
\sum_{(i,j,k,l) \in ijkl} w_{i,j,k,l}
$$

这个蛋白质比较问题的线性整数规划公式实际上是该问题的二次分配公式的线性化。使用Gurobi 9.1.0，您可以直接求解蛋白质比较问题的二次分配公式，无需辅助变量和逻辑约束。

### 目标函数
目标是最大化边匹配的数量。

$$
\sum_{(i,j,k,l) \in ijkl} x_{i,j}*x_{k,l}
$$

### 约束条件

**$G_{1}$ 匹配约束**：$G_{1}$ 中的节点最多只能与 $G_{2}$ 中的一个节点匹配。

$$
\sum_{i \in N_{1} } x_{i,j} \leq 1 \quad \forall j \in N_{2}
$$

**$G_{2}$ 匹配约束**：$G_{2}$ 中的节点最多只能与 $G_{1}$ 中的一个节点匹配。

$$
\sum_{j \in N_{2} } x_{i,j} \leq 1 \quad \forall i \in N_{1}
$$

**无交叉约束**：不能存在交叉。

$$
x_{i,j} + x_{k,l} \leq 1 \quad \forall 
(i,j,k,l) \in ijklx = \{ (i,j,k,l) \in ijkl: i < k \in N_{1}, j > l \in N_{2} \}
$$

## 输入数据

In [ ]:
# G1中的节点
nodes1 = [*range(1,10)]

# G1中的边(i,k)
edges1 = [(1,2),(2,9),(3,4),(3,5),(5,6),(6,7),(7,9),(8,9)]

# G2中的节点
nodes2 = [*range(1,12)]

# G2中的边(j,l)
edges2 = [(1,4),(2,3),(4,6),(4,7),(5,6),(6,8),(7,8),(7,10),(9,10),(10,11)]

## 预处理

In [ ]:
# 节点匹配：G1中的节点与G2中的节点的匹配
list_ij = []

for i in nodes1:
    for j in nodes2:
        tp = i,j
        list_ij.append(tp)
        
ij = gp.tuplelist(list_ij)

# 边匹配：G1中的边与G2中的边的匹配
list_ijkl = []

for i,k in edges1:
    for j,l in edges2:
        tp = i,j,k,l
        list_ijkl.append(tp)
        
ijkl = gp.tuplelist(list_ijkl)

# 无交叉
list_nox = []

for i,j in ij:
    for k,l in ij:
        if i < k and l < j:
            tp = i,j,k,l
            list_nox.append(tp)
            
nox = gp.tuplelist(list_nox)  

## 模型部署

我们创建一个模型和决策变量。决策变量映射每个图中的节点，并确保每个图的边被正确匹配。

In [ ]:
model = gp.Model('ProteinComparison')

# 将G1中的节点与G2中的节点进行映射
map_nodes = model.addVars(ij, vtype=GRB.BINARY, name="map")

Using license file c:\gurobi\gurobi.lic


**$G_{1}$ 匹配约束**：$G_{1}$ 中的节点最多只能与 $G_{2}$ 中的一个节点匹配。

$$
\sum_{i \in N_{1} } x_{i,j} \leq 1 \quad \forall j \in N_{2}
$$

In [ ]:
# G1中的一个节点最多只能与G2中的一个节点匹配
node1_match = model.addConstrs((gp.quicksum(map_nodes[i,j] for i in nodes1) <= 1 for j in nodes2 ) ,name='node1_match')

**$G_{2}$ 匹配约束**：$G_{2}$ 中的节点最多只能与 $G_{1}$ 中的一个节点匹配。

$$
\sum_{j \in N_{2} } x_{i,j} \leq 1 \quad \forall i \in N_{1}
$$

In [ ]:
# G2中的一个节点最多只能与G1中的一个节点匹配
node2_match = model.addConstrs((gp.quicksum(map_nodes[i,j] for j in nodes2) <= 1 for i in nodes1 ) ,name='node2_match')

**无交叉约束**：不能存在交叉。

$$
x_{i,j} + x_{k,l} \leq 1 \quad \forall 
(i,j,k,l) \in ijklx = \{ (i,j,k,l) \in ijkl: i < k \in N_{1}, j > l \in N_{2} \}
$$

In [ ]:
# 无交叉约束
no_crossover = model.addConstrs((map_nodes[i,j] + map_nodes[k,l] <= 1 for i,j,k,l in nox), name='no_crossover')

### 目标函数

最大化 G1 中的边与 G2 中的边的匹配数量。

$$
\sum_{(i,j,k,l) \in ijkl} x_{i,j}*x_{k,l}
$$

In [ ]:
# 目标函数
model.setObjective(gp.quicksum(map_nodes[i,j]*map_nodes[k,l] for i,j,k,l in ijkl ) , GRB.MAXIMIZE )

In [ ]:
# 验证模型构建
model.write('ProteinComparison.lp')

# 运行优化引擎
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2000 rows, 99 columns and 4158 nonzeros
Model fingerprint: 0x22958823
Model has 80 quadratic objective terms
Variable types: 0 continuous, 99 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1876 rows and 17 columns
Presolve time: 0.01s
Presolved: 204 rows, 162 columns, 2026 nonzeros
Variable types: 0 continuous, 162 integer (162 binary)

Root relaxation: objective -6.923077e+00, 178 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.92308    0   52   -0.00000    6.9230

In [ ]:
# 输出报告
print(f"Maximum number of edge matches: {round(model.objVal)}") 

for i,j,k,l in ijkl:
    if map_nodes[i,j].x*map_nodes[k,l].x > 0.5:
        print(f"Edge {i,k} in G1 is mapped with edge {j,l} in G2")

Maximum number of edge matches: 5
Edge (1, 2) in G1 is mapped with edge (2, 3) in G2
Edge (3, 4) in G1 is mapped with edge (4, 6) in G2
Edge (3, 5) in G1 is mapped with edge (4, 7) in G2
Edge (5, 6) in G1 is mapped with edge (7, 8) in G2
Edge (7, 9) in G1 is mapped with edge (9, 10) in G2


---
## 参考文献

H. Paul Williams, 《数学规划模型构建》，第五版。

Forrester, R.J. and Greenberg, H.J. (2008) 计算生物学中的二次二进制规划模型。算法运筹学，3，110–129。

版权所有 © 2020 Gurobi Optimization, LLC